# 复杂网络与社交网络分析课程设计

## 题目：K个影响力重要节点发现

Degree、PageRank、HITS、K-cores算法以及改进算法在数据集上的结果分析

## 使用的库

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
import numpy as np
from scipy import sparse
import math
import copy
import scipy as sp

## 读取 edgelist 文件并创建图

In [2]:
def read_graph_from_edgelist(file_path):
    # 读文件，制图
    f = open(file_path, 'r')
    # 建立有向图
    G = nx.DiGraph()
    
    lines = f.readlines()
    # 一次性读入整个文件，加快处理速度
    for line in lines:
        line = line.strip()     # 去掉结尾换行符
        line = line.split()     # 以空格或回车分割句子
        G.add_edge(int(line[0]), int(line[1]))
        # 建立边关系
    f.close()
    return G

## 重新加载已经存入txt的图

In [3]:
def load_graph_from_file(file_path):
    G = nx.read_edgelist(file_path,create_using=nx.DiGraph())
    return G

## 绘制度分布图

In [4]:
def draw_degree_plot(myGraph):
    # 获取图的度分布直方图
    Degree_Hist = nx.degree_histogram(myGraph)
    # 初始化一个列表，用于存储处理后的度分布数据
    Degree_Hist = [Degree_Hist, [], []]
    # 遍历度分布直方图，处理为 (度数, 频数) 的形式
    for i in range(0, len(Degree_Hist[0])):
        if Degree_Hist[0][i] == 0:
            continue
        Degree_Hist[1].append(i)    # 存储度数
        Degree_Hist[2].append(Degree_Hist[0][i])    # 存储对应的频数
     # 设置中文显示和字体
    plt.rcParams["font.sans-serif"] = "SimHei"
    plt.rcParams["axes.unicode_minus"] = False
    # 创建一个图形，设置图形大小
    fig = plt.figure(figsize=(12, 6))
     # 在图形中创建两个子图，分别绘制原始度数和对数化后的度数
    axes1 = plt.subplot(121)
    axes1.scatter([i for i in Degree_Hist[1]], [math.log(i + 1) for i in Degree_Hist[2]],
                  s=0.5, ) # 绘制原始度数的散点图
    axes1.set_xlabel("Degree")
    axes1.set_ylabel("log(count+1)")
    axes2 = plt.subplot(122)
    axes2.scatter([math.log(i) for i in Degree_Hist[1]], [math.log(i + 1) for i in Degree_Hist[2]],
                  s=0.8, )  # 绘制对数化后的度数的散点
    axes2.set_xlabel("log(Degree)")
    axes2.set_ylabel("log(count+1)")
    plt.suptitle("度分布散点图")
    plt.savefig("./pic/度分布散点图.png") # 保存图形到文件

## 清洗节点

In [5]:
def clean_node(data_file_name, G: nx.DiGraph):
    print('=' * 15 + f"数据清洗" + '=' * 15)
    f1 = './data/{}.txt'.format(data_file_name)
    f_clean = open(f1, 'w', encoding='utf-8')
    f2 = './data/higgs-{}_network.edgelist'.format(data_file_name)
    with open(f2, 'r', encoding='utf-8') as f:
        for line in f:
            l = line.split(' ')
            n1, n2 = int(l[0]), int(l[1])
            if n1 in G.nodes and n2 in G.nodes:
                f_clean.write(line)

## 统计边列表

In [6]:
#统计边列表,num表示图的节点数目,防止过大不方便绘制
def stat_node(data_file_name, num):
    f = './data/{}.txt'.format(data_file_name)
    edgeList = []
    count = 0
    with open(f, 'r', encoding='utf-8') as f:
        for line in f:
            if count >= num:
                return edgeList
            edge = [int(one) for one in line.strip('\n').split(' ')]
            edgeList.append(tuple(edge))
            count+=1
    return edgeList

## LT模块

In [7]:
__all__ = ['linear_threshold']

# -------------------------------------------------------------------------
#  Some Famous Diffusion Models
# -------------------------------------------------------------------------


def linear_threshold(G, seeds, steps=0):           # LT线性阈值算法
    """
      Parameters
      ----------
      G : networkx graph                     #所有节点构成的图
          The number of nodes.
      seeds: list of nodes                   #种子节点集
          The seed nodes of the graph
      steps: int                             #激活节点的层数（深度），当steps<=0时，返回子节点集能激活的所有节点
          The number of steps to diffuse
          When steps <= 0, the model diffuses until no more nodes
          can be activated
      Return
      ------
      layer_i_nodes : list of list of activated nodes
        layer_i_nodes[0]: the seeds                  #子节点集
        layer_i_nodes[k]: the nodes activated at the kth diffusion step   #该子节点集激活的节点集
      Notes
      -----
      1. Each node is supposed to have an attribute "threshold".  If not, the
         default value is given (0.5).    #每个节点有一个阈值，这里默认阈值为：0.5
      2. Each edge is supposed to have an attribute "influence".  If not, the
         default value is given (1/in_degree)  #每个边有一个权重值，这里默认为：1/入度
      References
      ----------
      [1] GranovetterMark. Threshold models of collective behavior.
          The American journal of sociology, 1978.
    """

    if type(G) == nx.MultiGraph or type(G) == nx.MultiDiGraph:
        raise Exception("linear_threshold() is not defined for graphs with multiedges.")

    # make sure the seeds are in the graph
    for s in seeds:
        if s not in G.nodes():
            raise Exception("seed", s, "is not in graph")

    # change to directed graph
    if not G.is_directed():
        DG = G.to_directed()
    else:
        DG = copy.deepcopy(G)        # copy.deepcopy 深拷贝 拷贝对象及其子对象

    # init thresholds
    for n in DG.nodes():
        if 'threshold' not in DG.nodes[n]:
            DG.nodes[n]['threshold'] = 0.5
        elif DG.nodes[n]['threshold'] > 1:
            raise Exception("node threshold:", DG.nodes[n]['threshold'], "cannot be larger than 1")

    #init influences
    in_deg = DG.in_degree()       # 获取所有节点的入度
    for e in DG.edges():
        if 'influence' not in DG[e[0]][e[1]]:
            DG[e[0]][e[1]]['influence'] = 1.0 / in_deg[e[1]]    # 计算边的权重
        elif DG[e[0]][e[1]]['influence'] > 1:
            raise Exception("edge influence:", DG[e[0]][e[1]]['influence'], "cannot be larger than 1")

    # perform diffusion
    A = copy.deepcopy(seeds)
    if steps <= 0:
        # perform diffusion until no more nodes can be activated
        return _diffuse_all(DG, A)
    # perform diffusion for at most "steps" rounds only
    return _diffuse_k_rounds(DG, A, steps)


def _diffuse_all(G, A):
    layer_i_nodes = []
    layer_i_nodes.append([i for i in A])
    while True:
        len_old = len(A)
        A, activated_nodes_of_this_round = _diffuse_one_round(G, A)
        layer_i_nodes.append(activated_nodes_of_this_round)
        if len(A) == len_old:
            break
    return layer_i_nodes


def _diffuse_k_rounds(G, A, steps):
    layer_i_nodes = []
    layer_i_nodes.append([i for i in A])
    while steps > 0 and len(A) < len(G):
        len_old = len(A)
        A, activated_nodes_of_this_round = _diffuse_one_round(G, A)
        layer_i_nodes.append(activated_nodes_of_this_round)
        if len(A) == len_old:
            break
        steps -= 1
    return layer_i_nodes


def _diffuse_one_round(G, A):
    activated_nodes_of_this_round = set()
    for s in A:
        nbs = G.successors(s)
        for nb in nbs:
            if nb in A:
                continue
            active_nb = list(set(G.predecessors(nb)).intersection(set(A)))
            if _influence_sum(G, active_nb, nb) >= G.nodes[nb]['threshold']:
                activated_nodes_of_this_round.add(nb)
    A.extend(list(activated_nodes_of_this_round))
    return A, list(activated_nodes_of_this_round)


def _influence_sum(G, froms, to):
    influence_sum = 0.0
    for f in froms:
        influence_sum += G[f][to]['influence']
    return influence_sum


## PageRank

In [8]:
#PageRank
def calculate_pagerank(graph):
    # 计算 PageRank 值
    #pagerank_scores = nx.pagerank(graph)
    pagerank_scores = func_pagerank(graph)
    return pagerank_scores

# 获取 PageRank 值最高的 k 个节点
def top_influential_nodes(scores):
    top_nodes = sorted(scores, key=scores.get, reverse=True)
    return top_nodes

def func_pagerank(G):
    alpha=0.85
    personalization=None
    max_iter=100
    tol=1.0e-6
    nstart=None
    weight="weight"
    dangling=None

    N = len(G)
    if N == 0:
        return {}
    nodelist = list(G)
    A = nx.to_scipy_sparse_array(G, nodelist=nodelist, weight=weight, dtype=float)
    S = A.sum(axis=1)
    S[S != 0] = 1.0 / S[S != 0]

    Q = sp.sparse.csr_array(sp.sparse.spdiags(S.T, 0, *A.shape))
    A = Q @ A
    # 初始向量
    if nstart is None:
        x = np.repeat(1.0 / N, N)
    else:
        x = np.array([nstart.get(n, 0) for n in nodelist], dtype=float)
        x /= x.sum()

    # 个性化向量
    if personalization is None:
        p = np.repeat(1.0 / N, N)
    else:
        p = np.array([personalization.get(n, 0) for n in nodelist], dtype=float)
        if p.sum() == 0:
            raise ZeroDivisionError
        p /= p.sum()
     # 悬空节点
    if dangling is None:
        dangling_weights = p
    else:
         # 将悬空节点的字典转换为按nodelist顺序的数组
        dangling_weights = np.array([dangling.get(n, 0) for n in nodelist], dtype=float)
        dangling_weights /= dangling_weights.sum()
    is_dangling = np.where(S == 0)[0]

    # 幂迭代：进行最多max_iter次迭代
    for _ in range(max_iter):
        xlast = x
        x = alpha * (x @ A + sum(x[is_dangling]) * dangling_weights) + (1 - alpha) * p
         # 检查收敛性，l1范数
        err = np.absolute(x - xlast).sum()
        if err < N * tol:
            return dict(zip(nodelist, map(float, x)))
    raise nx.PowerIterationFailedConvergence(max_iter)

#打印前100个得分最高的节点
def my_PageRank(graph):
     # 计算 PageRank 值
    pagerank_scores = calculate_pagerank(graph)
    # 获取 PageRank 值最高的 k 个节点
    top_nodes = top_influential_nodes(pagerank_scores)

    # # 设置要获取的影响力最大节点数
    # top_nodes_k = top_nodes[:100]
    # # 打印结果Top {k} Influential Nodes
    # f = open("PageRank.txt", 'w')
    # for node in top_nodes_k:
    #     f.write(f"Node:{node}  ,  PageRank Score:{pagerank_scores[node]}\n")

    return top_nodes

## HITS模块

In [9]:
#HITS模块

def calculate_hits(graph):
    # 计算 HITS 算法得到 Hub 和 Authority 值
    #hits_scores = nx.hits(graph)
    hits_scores = func_HITS(graph)
    return hits_scores

def func_HITS(G):
    max_iter=100
    tol=1.0e-8
    nstart=None
    normalized=True
    if len(G) == 0:
        return {}, {}
    A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)

    if nstart is None:
        _, _, vt = sp.sparse.linalg.svds(A, k=1, maxiter=max_iter, tol=tol)
    else:
        nstart = np.array(list(nstart.values()))
        _, _, vt = sp.sparse.linalg.svds(A, k=1, v0=nstart, maxiter=max_iter, tol=tol)

    a = vt.flatten().real
    h = A @ a
    if normalized:
        h /= h.sum()
        a /= a.sum()
    hubs = dict(zip(G, map(float, h)))
    authorities = dict(zip(G, map(float, a)))
    return hubs, authorities

def top_influential_nodes(scores):
    # 获取 Hub 或 Authority 值最高的 k 个节点
    top_nodes = sorted(scores, key=scores.get, reverse=True)
    return top_nodes

def my_HITS(graph):
    # 计算 HITS 算法得到 Hub 和 Authority 值
    hits_scores = calculate_hits(graph)
    # 获取 Authority 值最高的 k 个节点
    authority_nodes = top_influential_nodes(hits_scores[1])

    # 打印结果Top {k} Influential Nodes based on Authority:
    # top_authority_nodes = authority_nodes[:100]
    # f = open("HITS.txt", 'w')
    # for node in top_authority_nodes:
    #     f.write(f"Node:{node}  ,  Authority Score:{hits_scores[1][node]}\n")

    return authority_nodes

def get_HITS_Nodes(G):
    h,a=func_HITS(G)
    h1 = sorted(h.items(), key=itemgetter(1), reverse=True)  # 排序
    a1 = sorted(a.items(), key=itemgetter(1), reverse=True)  # 排序
    hhr = [e[0] for e in h1]   # 取节点
    har = [e[0] for e in a1]   # 取节点
    return hhr,har

## Kcores模块

In [10]:
import networkx as nx
import numpy as np
import scipy as sp
from operator import itemgetter

def K_cores(G, G2):
    # 通过各个节点的核数发现重要节点
    core = 1
    flag = 1
    node_cores = {}

    for node in G.nodes():
        node_cores[node] = -1

    while(G.number_of_nodes() != 0):
        # 循环去除
        while(flag == 1):
            # 还有变化
            flag = 0
            nodes = list(G.nodes())
            for node in nodes:
                if G.degree(node) <= core:
                    node_cores[node] = core
                    flag = 1
                    G.remove_node(node)
        flag = 1
        core += 1

    # print(node_cores)
    cores1 = sorted(node_cores.items(), key=itemgetter(1), reverse=True)  # 排序

    # ncores = cores1[:100]
    # f = open("K-coresRank.txt", 'w')
    # f.write("重要节点排序:\n" + str(ncores)+"\n\n各节点的K-coresRank值:\n" + str(cores1))
    # f.close()

    return cores1



## 度中心性模块

In [11]:
#度中心性模块

def degree_centrality(G):
    # 度中心性
    dc = nx.degree_centrality(G)  # degree
    dc1 = sorted(dc.items(), key=itemgetter(1), reverse=True)  # 排序

    # ndc = [e[0] for e in dc1[:100]]   # 取节点
    # f = open("DegreeRank.txt", 'w')
    # f.write("重要节点排序:\n" + str(ndc) + "\n\n各节点的度数中心性:\n" + str(dc1))
    # f.close()

    return dc1

## 卷积改进方法

In [12]:
def conv(G,atr):#atr是计算出来的degree
    conv={}#建立字典
    for edge in G.edges():
        if edge[0] in conv.keys():
            if edge[1] in atr.keys():
                conv[edge[0]] += atr[edge[1]] / atr[edge[0]]
        else:
            if edge[1] in atr.keys():
                conv[edge[0]] = atr[edge[1]] / atr[edge[0]]
            else:
                conv[edge[0]]=0
    return conv

## 计算单位时间内top-k节点的影响力范围

In [13]:
# 通过LT线性阈值模型，计算单位时间内top-k节点的影响力范围
def cal_influence(G, seeds, k=1): 
    layers = linear_threshold(G, seeds, k)      # seeds为种子节点集合
    length = 0
    for i in range(1, len(layers)):
        length += len(layers[i])        # 计算每个种子节点激活节点的个数之和
    return length

## 排序

In [14]:
def sort(dic):
    dic1 = sorted(dic.items(), key=itemgetter(1), reverse=True)  # 排序
    dnode = [e[0] for e in dic1]   # 取节点
    return dnode

## 预分析

In [15]:
def Primal_Info(myGraph: nx.Graph,fn):
    # 输出图的基本结构
    f = open("attribute_" + fn + ".txt", 'w')
    f.write("节点数: " + str(myGraph.number_of_nodes()))
    f.write("\n\n连边数: " + str(myGraph.number_of_edges()))
    f.write("\n\n网络密度: " + str(nx.density(myGraph)))
    f.write("\n\n平均网络的聚类系数: " + str(nx.average_clustering(nx.DiGraph(myGraph))))
    f.write("\n\n整个网络的聚类系数: " + str(nx.transitivity(myGraph)))
    f.close()

In [16]:
#主函数
myGraph = nx.read_edgelist("./data/higgs-social_network.edgelist",create_using=nx.DiGraph())

# compare_all_methods比较所有方法

In [17]:
#compare_all_methods比较所有方法
atr=dict(myGraph.out_degree())

conv1=conv(myGraph,atr)
convself=dict()
convself2=dict()
convself3=dict()
for k in atr.keys():
    if k in conv1.keys():
        convself2[k] = atr[k]+0.5*conv1[k]
        convself[k] = atr[k] + 0.2*conv1[k]
        convself3[k] = atr[k]+conv1[k]
    else:
        convself2[k] = atr[k]
        convself[k] = atr[k]
        convself3[k] = atr[k]
# 输出图的基本结构
k = 10
x = []
y1 = []
y2 = []
y3 = []
y4=[]
y5=[]
y6=[]
y7=[]
y8=[]

PageRank_nodes=my_PageRank(myGraph)
HITS_hubs,HITS_authorities = get_HITS_Nodes(myGraph)
myGraph_copy = myGraph.copy()
kcores = K_cores(myGraph_copy, myGraph)  
degree_centality = degree_centrality(myGraph)

nc1=sort(convself)
nc2=sort(convself2)
nc3=sort(convself3)


In [ ]:
 for seeds_num in range(200, 410,50):
    seed_node = str(seeds_num)
    if seed_node not in myGraph:
        print(f"Seed node {seed_node} is not in the graph.")
        continue 
    print('='*20+seed_node+"\n")
    x.append(seed_node)

    #print("degree_centality[1]={}\n".format(degree_centality[1]))
    degree_centality_seeds = [e[0] for e in degree_centality[:seeds_num]] 

    #print("kcores[1]={}\n".format(kcores[1]))
    kcores_seeds = [k[0] for k in kcores[:seeds_num]] 

    #print("PageRank_nodes[1]={}\n".format(PageRank_nodes[1]))
    PageRank_nodes_seeds = [p[0] for p in PageRank_nodes[:seeds_num]] 

    #print("HITS_hubs[1]={}\n".format(HITS_hubs[1]))
    HITS_hubs_seeds = [h[0] for h in HITS_hubs[:seeds_num]] 

    #print("HITS_authorities[1]={}\n".format(HITS_authorities[1]))
    HITS_authorities_seeds = [a[0] for a in HITS_authorities[:seeds_num]] 

    #print("nc1[1]={}\n".format(nc1[1]))
    nc1_seeds = [n[0] for n in nc1[:seeds_num]] 
    nc2_seeds = [n[0] for n in nc2[:seeds_num]] 
    nc3_seeds = [n[0] for n in nc3[:seeds_num]] 

    y1.append(cal_influence(myGraph, degree_centality_seeds, k))
    y2.append(cal_influence(myGraph, kcores_seeds, k))
    y3.append(cal_influence(myGraph, nc1_seeds, k))
    y4.append(cal_influence(myGraph, nc2_seeds, k))
    y5.append(cal_influence(myGraph, PageRank_nodes_seeds, k))
    y6.append(cal_influence(myGraph, HITS_hubs_seeds, k))
    y7.append(cal_influence(myGraph, HITS_authorities_seeds, k))
    y8.append(cal_influence(myGraph, nc3_seeds, k))

In [ ]:
l1 = plt.plot(x, y1, 'r--', label='Degree')
l2 = plt.plot(x, y2, 'c--', label='Kcores')
l3 = plt.plot(x, y3, 'b--', label='Conv0.2')
l4 = plt.plot(x, y4, 'k--', label='Conv0.5')
l8 = plt.plot(x, y8, 'y--', label='Conv1')
l5 = plt.plot(x, y5, 'g--', label='Pagerank')
l6 = plt.plot(x, y6, 'm--', label='HITS-H')
l7 = plt.plot(x, y7, 'm--', label='HITS-A')
print("end")
plt.plot(x, y1, 'ro-', x, y2, 'c+-', x, y3, 'b^-', x, y4, 'ks-' , x, y5, 'gp-', x, y6, 'm1-', x, y7, 'm2-', x, y8, 'y3-')
plt.legend()
plt.savefig('Compare.png')
plt.show()